In [ ]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

data_path = os.path.join('..','Data')

pd.options.display.float_format = '{:.3f}'.format

### Load Input-Output data

In [ ]:
Rijnmond_IO = pd.read_csv(os.path.join(data_path,'Rijnmond_IO.csv'),header=[0],index_col=[0])

### Create mapping dictionary so we have names instead of codes

In [ ]:
sector_codes = ['A','B','C1','C2','C3','C4','C5','C6','C7','C8','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T']
full_sector_names = ['Agriculture, forestry and fishing','Quarrying',
'General Industry','Food industry','Oil refinery','Chemical & biobased industry','Basic metal-metal products industry',
'Means of transport ind. and maritime service','Electricity generation','Other Industry',
'Construction industry','Trade','Transport and storage','Catering industry',
'Information and communication','Financial services','Rental and trading of real estate',
'Specialist Business Services','Rental and other business services','Public administration and government services',
'Education','Health and welfare care','Culture, sport and recreation','Other services','Household services']

sector_dict = dict(zip(sector_codes,full_sector_names))

### And change the index and column names 

In [ ]:
Rijnmond_IO = Rijnmond_IO.rename(columns=sector_dict,index=sector_dict)

### Create A matrix

In [ ]:
A = np.array(Rijnmond_IO.iloc[:25,:25])/np.array(Rijnmond_IO.iloc[:,:25].sum(axis=0))
A = np.nan_to_num(A)

### Create I Matrix

In [ ]:
I = np.diag((np.ones(25)))

### Create Leontief Inverse
To create the inverse of a matrix, we can use the function np.linalg.inv(x)

In [ ]:
LI = np.linalg.inv((I-A))

### Test the Leontief Inverse

In [ ]:
TFD = Rijnmond_IO.iloc[:25,25:].sum(axis=1) # total final demand (household demand and exports)

In [ ]:
X = np.dot(LI,TFD) # multiply the leontief inverse with the total final demand

In [ ]:
# compare X with the sector totals, check if it is correct



### Estimate impacts of a reduction of 50% in demand for food products
We first need to get the Total Final Demand as pandas Series, so we know which value belongs to which sector.

In [ ]:
NEW_TFD_FOOD = TFD.copy() # Create a copy of the total final demand, where we can change the value
NEW_TFD_FOOD['Food industry'] = NEW_TFD_FOOD['Food industry']*0.5 # and reduce the food industry by 50% 

Use the function np.dot() and have a look at the slides on how to perform this

In [ ]:
X_new_food = np.dot(LI,NEW_TFD_FOOD)

And calculate the difference per sector:

In [ ]:
loss_per_sector_food = pd.DataFrame(X_new_food-X,index=TFD.index,columns=['losses'])

And the total losses:

### And do the same for financial services

In [ ]:
NEW_TFD_FIN = TFD.copy() # Create a copy of the total final demand, where we can change the value
NEW_TFD_FIN['Financial services'] = TFD['Financial services']*0.5 # and reduce the food industry by 50% 

Use the function np.dot() and have a look at the slides on how to perform this

In [ ]:
X_new_fin = np.dot(LI,NEW_TFD_FIN)

And calculate the difference per sector:

In [ ]:
loss_per_sector_fin = pd.DataFrame(X_new_fin-X,index=TFD.index,columns=['losses'])

And the total losses:

### Finally, let's use the Leontief Inverse to calculate the losses due to the flood

We need to first load the direct damage

In [ ]:
output_path = os.path.join(data_path,'..','output')
damages_sector = pd.read_csv(os.path.join(output_path,'damages_sector.csv'),header=[0],index_col=[0])
damages_sector = damages_sector.rename(index=sector_dict)

And translate direct impacts to relative impacts

In [ ]:
ValueA = Rijnmond_IO.iloc[25,:25]*1e6

In [ ]:
damages_sector = damages_sector.merge(ValueA,left_index=True,right_index=True)
damages_sector['disruption'] = damages_sector['damages'].div(damages_sector.Z_BrTW)
damages_sector['disruption'] = 1-damages_sector['disruption']
damages_sector = damages_sector.drop('Z_BrTW',axis=1)

# And now merge the two Series so we have a full list of industries and their relative reduction
disr_per_sector = pd.merge(pd.DataFrame(damages_sector['disruption']), pd.DataFrame(TFD,columns=['TFD']), 
                            left_index=True,right_index=True, how='right').fillna(1)

Now we can get the new total final demand, based on the disruption as a result of the flood

In [ ]:
NEW_TFD_FLOOD = disr_per_sector['disruption']*disr_per_sector['TFD']

Use the function np.dot() again

In [ ]:
X_new_flood  = np.dot(LI,NEW_TFD_FLOOD)

And calculate the difference per sector:

In [ ]:
loss_per_sector_flood = pd.DataFrame(X_new_flood-X,index=TFD.index,columns=['losses'])

And the total losses:

To be able to compare the losses with the output from the ARIO model, we need to translate these total production losses to losses in terms of Value Added:

In [ ]:
loss_value_added = loss_per_sector_flood.multiply((Rijnmond_IO.iloc[25,:25]/X),axis=0)

In [ ]:
loss_value_added